In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import *
from sbm_class import *
from neal_batched import *
from neal_sequential import *
from batched import *
from sequential import *
from metrics import *

## Assessing Convergence

**Approach:**

We implement a convergence criterion based on the one proposed in the book "Bayesian Data Analysis" by Gelman and Carlin, the $\hat{R}$ of page 285 (equation 11.4).

Let us take situations with a strong recovery, and we measure for different $\gamma$'s the convergence speed. We repeat this for networks increasing in size (notice that, by our proof, we are concerned with $\frac{n}{k}$, not only $n$, so we may want to test if increasing the community size is relevant or not).

In [2]:
def split_chains(chains, splits = 2):
    # chains is a list of arrays
    n_tot = len(chains[0])
    if n_tot % splits != 0:
        raise ValueError("The number of samples is not divisible by the number of splits")
    # split each chain into splits parts
    split_chains = []
    for chain in chains:
        split_chain = np.array_split(chain, splits)
        split_chains.append(split_chain)
    return np.vstack(split_chains).T # row index: iteration, col_index: chain

def R_hat(split_chains):
    n = split_chains.shape[0]
    m = split_chains.shape[1]
    # within chains variance
    W = np.mean(np.sum((split_chains - np.mean(split_chains, axis=0))**2, axis=0)/(n-1))
    # between chains variance
    B = n*np.sum((np.mean(split_chains, axis=0) - np.mean(split_chains))**2)/(m-1)
    # estimated variance
    var_plus = (n-1)/n*W + 1/n*B
    return np.sqrt(var_plus/W)

In [3]:
# initialization parameter
alpha = 0.2
# set parameters for the network
n = 400
k = 4
p = 0.17
q = 0.08
# set parameters for the samplers
n_iter = 250
burn_in = 50

alpha_p = 1
beta_p = 1
alpha_q = 1
beta_q = 1

# changing parameters
pi = np.ones(k) # deterministic prior
gamma_list = [0, 0.1, 1, 10, 10000, 1000000]

# seed
np.random.seed(0)

splits = 2
n_chains = 4

In [4]:
sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
A = sbm.get_A()
z = sbm.get_z()
# warm initialization


In [5]:
def progressive_R_hat(estimand_array, splits = 2, n_chains = 4, n_iter=250, burn_in=0, ignore=0):

    # compute R_hat for subsequence (1,r) for all r
    R_hat_list = np.zeros(n_iter-burn_in//splits)

    if not isinstance(estimand_array, np.ndarray):
        estimand_array = np.array(estimand_array).T

    for r in range(1+splits, n_iter-burn_in, splits):
        l = int(r*ignore)
        while ((r+1-l) % splits) != 0:
            l = l - 1
        # take the first r rows, possibly ignoring a portion of the samples
        list_split = estimand_array[l:r+1, :]
        # convert list_split from array to list of arrays (n_chains arrays)
        list_split = [list_split[:, i] for i in range(n_chains)]
        list_split = split_chains(list_split, splits)
        R_hat_list[r//splits] = R_hat(list_split)
        return R_hat_list

In [6]:
extra = False

In [7]:
 if extra:
    # run the samplers (n_chains)
    # batched
    batched_p_list = np.zeros((n_iter, n_chains))
    batched_q_list = np.zeros((n_iter, n_chains))
    for a in range(n_chains):
        z_init = warm_initializer(z, alpha, n, k)
        sampler = batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, pi_pri = pi)
        sampler.run(n_iter)
        # store the p_lists
        batched_p_list[:, a] = sampler.get_p_list()
        batched_q_list[:, a] = sampler.get_q_list()

    # compute R_hat for subsequence (1,r) for all r
    batched_R_hat_p = np.zeros(n_iter//2)
    batched_R_hat_q = np.zeros(n_iter//2)

    for r in range(1+splits, n_iter, 2):
        # take the first r rows of the p_list and q_list
        p_list_split = batched_p_list[:r+1, :]
        q_list_split = batched_q_list[:r+1, :]
        # convert p_list_split from array to list of arrays (n_chains arrays)
        p_list_split = [p_list_split[:, i] for i in range(n_chains)]
        q_list_split = [q_list_split[:, i] for i in range(n_chains)]
        p_list_split = split_chains(p_list_split, splits)
        q_list_split = split_chains(q_list_split, splits)
        batched_R_hat_p[r//2] = R_hat(p_list_split)
        batched_R_hat_q[r//2] = R_hat(q_list_split)

    # plot the R_hat for p and q in separate subplots
    plt.figure(figsize = (15,15))
    plt.subplot(2, 1, 1)
    plt.plot(batched_R_hat_p[3:], label='R_hat_p')
    plt.title('R_hat for p')
    plt.subplot(2, 1, 2)
    plt.plot(batched_R_hat_q[3:], label='R_hat_q')
    plt.title('R_hat for q')
    plt.show()


We need to define a convergence threshold, say $\hat{R}=0.99$.


In [8]:
pic = False

In [9]:
if extra:
    # batched sampler
    # run the samplers (n_chains)
    # batched
    batched_p_list = np.zeros((n_iter, n_chains))
    batched_q_list = np.zeros((n_iter, n_chains))
    loss_list = np.zeros((n_iter, n_chains))
    for a in range(n_chains):
        z_init = warm_initializer(z, alpha, n, k)
        sampler = batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, pi_pri = pi)
        sampler.run(n_iter)
        # store the p_lists
        batched_p_list[:, a] = sampler.get_p_list()
        batched_q_list[:, a] = sampler.get_q_list()
        z_list = sampler.get_z_list()
        loss_list[:, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

    # compute R_hat for subsequence (1,r) for all r
    batched_R_hat_p = np.zeros(n_iter//2)
    batched_R_hat_q = np.zeros(n_iter//2)
    idx = np.zeros(n_iter//2)

    for r in range(1+splits, n_iter, 2):
        # take the first r rows of the p_list and q_list
        p_list_split = batched_p_list[:r+1, :]
        q_list_split = batched_q_list[:r+1, :]
        # convert p_list_split from array to list of arrays (n_chains arrays)
        p_list_split = [p_list_split[:, i] for i in range(n_chains)]
        q_list_split = [q_list_split[:, i] for i in range(n_chains)]
        p_list_split = split_chains(p_list_split, splits)
        q_list_split = split_chains(q_list_split, splits)
        batched_R_hat_p[r//2] = R_hat(p_list_split)
        batched_R_hat_q[r//2] = R_hat(q_list_split)
        idx[r//2] = r

    # average the loss over the chains
    loss_avg = np.mean(loss_list, axis=1)

    # plot the R_hat for p and q in the same subplot, and the loss in a separate subplot
    plt.figure(figsize = (15,15))
    plt.subplot(2, 1, 1)
    plt.plot(idx[3:], batched_R_hat_p[3:], label='R_hat_p')
    plt.plot(idx[3:], batched_R_hat_q[3:], label='R_hat_q')
    plt.title('R_hat for p and q')
    plt.subplot(2, 1, 2)
    plt.plot(loss_avg, label='loss')
    plt.title('loss')
    if pic:
        plt.savefig('r_hat/batched_R_hat_loss.png')
    plt.show()


    # all gamma values

In [10]:
def find_convergence(list, lb, ub):
    candidates = []
    for i in range(len(list)):
        if list[i] > lb and list[i] < ub:
            candidates.append(i)
    for i in candidates:
        # check if all the following values are within the bounds
        if all([list[j] > lb and list[j] < ub for j in range(i+1, len(list))]):
            return i
    return -1

## Grid Code

In [37]:
# set parameters for the network
n = 500
k = 4
# set parameters for the samplers
n_iter = 50
burn_in = 0

alpha_p = 1
beta_p = 1
alpha_q = 1
beta_q = 1

seed = 1

# changing parameters
pi = np.ones((n, k)) # deterministic prior
# row normalization of pi
pi = pi/pi.sum(axis=1)[:, None]
gamma_list = [0, 0.1, 1, 10, 10000]

param_list = [(0.17, 0.08, 0.2), (0.4, 0.2, 0.4)] #(p,q,alpha)
# for each value in the list, we should assess convergence using the r_hat values on the loss
# we do it for the batched sampler, the sequential one, the neal sequential ones wiht different gamma values and the neal batched ones with 
# different gamma values
# we will store the values of r_hat for the loss
splits = 2
n_chains = 4

lb = 0.925
ub = 1.075

In [38]:
df_batched = pd.DataFrame(columns=['p', 'q', 'alpha', 'convergence_loss', 'convergence_p', 'convergence_q'])

np.random.seed(seed)
pic = False

# run the samplers (n_chains)
# batched
batched_p_list = np.zeros((len(param_list), n_iter, n_chains))
batched_q_list = np.zeros((len(param_list), n_iter, n_chains))
loss_list = np.zeros((len(param_list), n_iter, n_chains))

for (p,q,alpha) in param_list:
    sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
    A = sbm.get_A()
    z = sbm.get_z()
    for a in range(n_chains):
        z_init = warm_initializer(z, alpha, n, k)
        sampler = batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, pi_pri = pi)
        sampler.run(n_iter)
        # store the p_lists
        batched_p_list[param_list.index((p,q,alpha)), :, a] = sampler.get_p_list()
        batched_q_list[param_list.index((p,q,alpha)), :, a] = sampler.get_q_list()
        z_list = sampler.get_z_list()
        loss_list[param_list.index((p,q,alpha)), :, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

    # compute R_hat for subsequence (1,r) for all r
    batched_R_hat_p = np.zeros(n_iter//2)
    batched_R_hat_q = np.zeros(n_iter//2)
    batched_R_hat_loss = np.zeros(n_iter//2)
    idx = np.zeros(n_iter//2)

    for r in range(1+splits, n_iter, 2):
        # take the first r rows of the p_list and q_list
        p_list_split = batched_p_list[param_list.index((p,q,alpha)),:r+1, :]
        q_list_split = batched_q_list[param_list.index((p,q,alpha)),:r+1, :]
        loss_list_split = loss_list[param_list.index((p,q,alpha)),:r+1, :]
        # convert p_list_split from array to list of arrays (n_chains arrays)
        p_list_split = [p_list_split[:, i] for i in range(n_chains)]
        q_list_split = [q_list_split[:, i] for i in range(n_chains)]
        loss_list_split = [loss_list_split[:, i] for i in range(n_chains)]
        p_list_split = split_chains(p_list_split, splits)
        q_list_split = split_chains(q_list_split, splits)
        loss_list_split = split_chains(loss_list_split, splits)
        batched_R_hat_p[r//2] = R_hat(p_list_split)
        batched_R_hat_q[r//2] = R_hat(q_list_split)
        batched_R_hat_loss[r//2] = R_hat(loss_list_split)
        idx[r//2] = r

    # average the loss over the chains
    # loss_avg = np.mean(loss_list[param_list.index((p,q,alpha)), :, :], axis=1)
    # print(loss_avg)

    # plot the R_hat for p and q in the same subplot, and the loss in a separate subplot
    # plt.figure(figsize = (15,15))
    # plt.subplot(2, 1, 1)
    # plt.plot(idx[3:], batched_R_hat_p[3:], label='R_hat_p')
    # plt.plot(idx[3:], batched_R_hat_q[3:], label='R_hat_q')
    # plt.title('R_hat for p and q')
    # plt.subplot(2, 1, 2)
    # plt.plot(batched_R_hat_loss[3:], label='loss')
    # plt.title('loss')
    # plt.show()

    # compute the convergence
    conv_loss = find_convergence(batched_R_hat_loss, lb, ub)
    conv_p = find_convergence(batched_R_hat_p, lb, ub)
    conv_q = find_convergence(batched_R_hat_q, lb, ub)

    print(f'Convergence for p={p}, q={q}, alpha={alpha}: loss={conv_loss}, p={conv_p}, q={conv_q}')

    tmp = pd.DataFrame({'p': p, 'q': q, 'alpha': alpha, 'convergence_loss': conv_loss, 'convergence_p': conv_p, 'convergence_q': conv_q}, index = [0])

    df_batched = pd.concat([df_batched, tmp], axis=0)
    

100%|██████████| 50/50 [00:00<00:00, 93.84it/s]
C:\Users\feder\AppData\Local\Temp\ipykernel_44752\1691313796.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_batched = pd.concat([df_batched, tmp], axis=0)


Convergence for p=0.17, q=0.08, alpha=0.2: loss=20, p=11, q=17


100%|██████████| 50/50 [00:00<00:00, 91.73it/s]

Convergence for p=0.4, q=0.2, alpha=0.4: loss=4, p=5, q=4


In [39]:
df_batched

,p,q,alpha,convergence_loss,convergence_p,convergence_q
0,0.17,0.08,0.2,20,11,17
0,0.40,0.20,0.4,4,5,4


In [40]:
# repeat with sequential sampler
df_sequential = pd.DataFrame(columns=['p', 'q', 'alpha', 'convergence_loss', 'convergence_p', 'convergence_q'])

np.random.seed(seed)

# run the samplers (n_chains)

sequential_p_list = np.zeros((len(param_list), n_iter, n_chains))
sequential_q_list = np.zeros((len(param_list), n_iter, n_chains))
sequential_loss_list = np.zeros((len(param_list), n_iter, n_chains))

for (p,q,alpha) in param_list:
    sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
    A = sbm.get_A()
    z = sbm.get_z()
    for a in range(n_chains):
        z_init = warm_initializer(z, alpha, n, k)
        sampler = sequential_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, pi_pri = pi)
        sampler.run(n_iter)
        # store the p_lists
        sequential_p_list[param_list.index((p,q,alpha)), :, a] = sampler.get_p_list()
        sequential_q_list[param_list.index((p,q,alpha)), :, a] = sampler.get_q_list()
        sequential_loss_list[param_list.index((p,q,alpha)), :, a] = [loss(z, sampler.get_z_list()[i])[0] for i in range(n_iter)]

    # compute R_hat for subsequence (1,r) for all r
    sequential_R_hat_p = np.zeros(n_iter//2)
    sequential_R_hat_q = np.zeros(n_iter//2)
    sequential_R_hat_loss = np.zeros(n_iter//2)
    idx = np.zeros(n_iter//2)

    for r in range(1+splits, n_iter, 2):
        # take the first r rows of the p_list and q_list
        p_list_split = sequential_p_list[param_list.index((p,q,alpha)),:r+1, :]
        q_list_split = sequential_q_list[param_list.index((p,q,alpha)),:r+1, :]
        loss_list_split = sequential_loss_list[param_list.index((p,q,alpha)),:r+1, :]
        # convert p_list_split from array to list of arrays (n_chains arrays)
        p_list_split = [p_list_split[:, i] for i in range(n_chains)]
        q_list_split = [q_list_split[:, i] for i in range(n_chains)]
        loss_list_split = [loss_list_split[:, i] for i in range(n_chains)]
        p_list_split = split_chains(p_list_split, splits)
        q_list_split = split_chains(q_list_split, splits)
        loss_list_split = split_chains(loss_list_split, splits)
        sequential_R_hat_p[r//2] = R_hat(p_list_split)
        sequential_R_hat_q[r//2] = R_hat(q_list_split)
        sequential_R_hat_loss[r//2] = R_hat(loss_list_split)
        idx[r//2] = r

    # compute the convergence
    conv_loss = find_convergence(sequential_R_hat_loss, lb, ub)
    conv_p = find_convergence(sequential_R_hat_p, lb, ub)
    conv_q = find_convergence(sequential_R_hat_q, lb, ub)


    # # plot the R_hat for p and q in the same subplot, and the loss in a separate subplot
    # plt.figure(figsize = (15,15))
    # plt.subplot(2, 1, 1)
    # plt.plot(idx[3:], sequential_R_hat_p[3:], label='R_hat_p')
    # plt.plot(idx[3:], sequential_R_hat_q[3:], label='R_hat_q')
    # plt.title('R_hat for p and q')
    # plt.subplot(2, 1, 2)
    # plt.plot(sequential_R_hat_loss[3:], label='loss')
    # plt.title('loss')
    # plt.show()

    print(f'Convergence for p={p}, q={q}, alpha={alpha}: loss={conv_loss}, p={conv_p}, q={conv_q}')

    tmp = pd.DataFrame({'p': p, 'q': q, 'alpha': alpha, 'convergence_loss': conv_loss, 'convergence_p': conv_p, 'convergence_q': conv_q}, index = [0])

    df_sequential = pd.concat([df_sequential, tmp], axis=0)


100%|██████████| 50/50 [00:01<00:00, 32.38it/s]
C:\Users\feder\AppData\Local\Temp\ipykernel_44752\1130406705.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_sequential = pd.concat([df_sequential, tmp], axis=0)


Convergence for p=0.17, q=0.08, alpha=0.2: loss=2, p=5, q=3


100%|██████████| 50/50 [00:01<00:00, 31.71it/s]


Convergence for p=0.4, q=0.2, alpha=0.4: loss=2, p=3, q=3


In [41]:
# repeat the same but for the neal sequential sampler
df_neal_seq = pd.DataFrame(columns=['p', 'q', 'alpha', 'gamma', 'convergence_loss', 'convergence_p', 'convergence_q'])

pic = False

# run the samplers (n_chains)
# batched
neal_seq_p_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))
neal_seq_q_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))  
loss_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))



for gamma in gamma_list:
    np.random.seed(seed)
    for (p,q,alpha) in param_list:
        sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
        A = sbm.get_A()
        z = sbm.get_z()
        for a in range(n_chains):
            z_init = warm_initializer(z, alpha, n, k)
            sampler = Neal_sequential_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, gamma=gamma)
            sampler.run(n_iter)
            # store the p_lists
            neal_seq_p_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = sampler.get_p_list()
            neal_seq_q_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = sampler.get_q_list()
            z_list = sampler.get_z_list()
            loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

        # compute R_hat for subsequence (1,r) for all r
        neal_seq_R_hat_p = np.zeros(n_iter//2)
        neal_seq_R_hat_q = np.zeros(n_iter//2)
        neal_seq_R_hat_loss = np.zeros(n_iter//2)
        idx = np.zeros(n_iter//2)

        for r in range(1+splits, n_iter, 2):
            # take the first r rows of the p_list and q_list
            p_list_split = neal_seq_p_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            q_list_split = neal_seq_q_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            loss_list_split = loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            # convert p_list_split from array to list of arrays (n_chains arrays)
            p_list_split = [p_list_split[:, i] for i in range(n_chains)]
            q_list_split = [q_list_split[:, i] for i in range(n_chains)]
            loss_list_split = [loss_list_split[:, i] for i in range(n_chains)]
            p_list_split = split_chains(p_list_split, splits)
            q_list_split = split_chains(q_list_split, splits)
            loss_list_split = split_chains(loss_list_split, splits)
            neal_seq_R_hat_p[r//2] = R_hat(p_list_split)
            neal_seq_R_hat_q[r//2] = R_hat(q_list_split)
            neal_seq_R_hat_loss[r//2] = R_hat(loss_list_split)
            idx[r//2] = r

        # average the loss over the chains
        loss_avg = np.mean(loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, :], axis=1)

        # compute the convergence
        conv_loss = find_convergence(neal_seq_R_hat_loss, lb, ub)
        conv_p = find_convergence(neal_seq_R_hat_p, lb, ub)
        conv_q = find_convergence(neal_seq_R_hat_q, lb, ub)

        print(f'Convergence for p={p}, q={q}, alpha={alpha}, gamma={gamma}: loss={conv_loss}, p={conv_p}, q={conv_q}')

        tmp = pd.DataFrame({'p': p, 'q': q, 'alpha': alpha, 'gamma': gamma, 'convergence_loss': conv_loss, 'convergence_p': conv_p, 'convergence_q': conv_q}, index = [0])
        df_neal_seq = pd.concat([df_neal_seq, tmp], axis=0)


100%|██████████| 50/50 [00:01<00:00, 25.21it/s]
C:\Users\feder\AppData\Local\Temp\ipykernel_44752\2576818073.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_neal_seq = pd.concat([df_neal_seq, tmp], axis=0)


Convergence for p=0.17, q=0.08, alpha=0.2, gamma=0: loss=7, p=5, q=3


100%|██████████| 50/50 [00:01<00:00, 31.09it/s]


Convergence for p=0.4, q=0.2, alpha=0.4, gamma=0: loss=2, p=3, q=3


100%|██████████| 50/50 [00:01<00:00, 32.22it/s]


...

Convergence for p=0.4, q=0.2, alpha=0.4, gamma=10: loss=2, p=3, q=3


100%|██████████| 50/50 [00:01<00:00, 30.95it/s]


Convergence for p=0.17, q=0.08, alpha=0.2, gamma=10000: loss=5, p=5, q=3


100%|██████████| 50/50 [00:01<00:00, 31.60it/s]

Convergence for p=0.4, q=0.2, alpha=0.4, gamma=10000: loss=2, p=3, q=3


In [16]:
# repeat with the neal batched sampler

df_neal_batched = pd.DataFrame(columns=['p', 'q', 'alpha', 'gamma', 'convergence_loss', 'convergence_p', 'convergence_q'])

pic = False

# run the samplers (n_chains)
# batched
neal_batched_p_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))
neal_batched_q_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))
loss_list = np.zeros((len(gamma_list), len(param_list), n_iter, n_chains))

for gamma in gamma_list:
    np.random.seed(seed)
    for (p,q,alpha) in param_list:
        sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
        A = sbm.get_A()
        z = sbm.get_z()
        for a in range(n_chains):
            z_init = warm_initializer(z, alpha, n, k)
            sampler = Neal_batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, gamma=gamma)
            sampler.run(n_iter)
            # store the p_lists
            neal_batched_p_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = sampler.get_p_list()
            neal_batched_q_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = sampler.get_q_list()
            z_list = sampler.get_z_list()
            loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

        # compute R_hat for subsequence (1,r) for all r
        neal_batched_R_hat_p = np.zeros(n_iter//2)
        neal_batched_R_hat_q = np.zeros(n_iter//2)
        neal_batched_R_hat_loss = np.zeros(n_iter//2)
        idx = np.zeros(n_iter//2)

        for r in range(1+splits, n_iter, 2):
            # take the first r rows of the p_list and q_list
            p_list_split = neal_batched_p_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            q_list_split = neal_batched_q_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            loss_list_split = loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :r+1, :]
            # convert p_list_split from array to list of arrays (n_chains arrays)
            p_list_split = [p_list_split[:, i] for i in range(n_chains)]
            q_list_split = [q_list_split[:, i] for i in range(n_chains)]
            loss_list_split = [loss_list_split[:, i] for i in range(n_chains)]
            p_list_split = split_chains(p_list_split, splits)
            q_list_split = split_chains(q_list_split, splits)
            loss_list_split = split_chains(loss_list_split, splits)
            neal_batched_R_hat_p[r//2] = R_hat(p_list_split)
            neal_batched_R_hat_q[r//2] = R_hat(q_list_split)
            neal_batched_R_hat_loss[r//2] = R_hat(loss_list_split)
            idx[r//2] = r

        # average the loss over the chains
        loss_avg = np.mean(loss_list[gamma_list.index(gamma), param_list.index((p,q,alpha)), :], axis=1)

        # compute the convergence
        conv_loss = find_convergence(neal_batched_R_hat_loss, lb, ub)
        conv_p = find_convergence(neal_batched_R_hat_p, lb, ub)
        conv_q = find_convergence(neal_batched_R_hat_q, lb, ub)

        print(f'Convergence for p={p}, q={q}, alpha={alpha}, gamma={gamma}: loss={conv_loss}, p={conv_p}, q={conv_q}')

        tmp = pd.DataFrame({'p': p, 'q': q, 'alpha': alpha, 'gamma': gamma, 'convergence_loss': conv_loss, 'convergence_p': conv_p, 'convergence_q': conv_q}, index = [0])
        df_neal_batched = pd.concat([df_neal_batched, tmp], axis=0)
        

100%|██████████| 50/50 [00:00<00:00, 102.60it/s]
C:\Users\feder\AppData\Local\Temp\ipykernel_44752\2954739488.py:63: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_neal_batched = pd.concat([df_neal_batched, tmp], axis=0)


Convergence for p=0.17, q=0.08, alpha=0.3, gamma=0: loss=13, p=10, q=9


100%|██████████| 50/50 [00:00<00:00, 114.88it/s]


Convergence for p=0.4, q=0.2, alpha=0.4, gamma=0: loss=7, p=6, q=6


100%|██████████| 50/50 [00:00<00:00, 92.48it/s]


...

Convergence for p=0.4, q=0.2, alpha=0.4, gamma=10: loss=7, p=6, q=6


100%|██████████| 50/50 [00:00<00:00, 110.24it/s]


Convergence for p=0.17, q=0.08, alpha=0.3, gamma=10000: loss=13, p=10, q=9


100%|██████████| 50/50 [00:00<00:00, 113.44it/s]

Convergence for p=0.4, q=0.2, alpha=0.4, gamma=10000: loss=7, p=6, q=6


In [25]:
# combine dataframes
df_batched['sampler'] = 'batched'
df_sequential['sampler'] = 'sequential'
df_neal_seq['sampler'] = 'neal_seq'
df_neal_batched['sampler'] = 'neal_batched'

# add a null gamma row to batched and sequential
df_batched['gamma'] = None
df_sequential['gamma'] = None

df = pd.concat([df_batched, df_sequential, df_neal_seq, df_neal_batched], axis=0)
# filter the alpha = 0.2
df_weak = df[df['alpha'] == 0.3]
df_strong = df[df['alpha'] == 0.4]

# sort them by the average of the convergence
avg = df_weak[['convergence_loss', 'convergence_p', 'convergence_q']].mean(axis = 1)
df_weak["avg"] = avg
avg = df_strong[['convergence_loss', 'convergence_p', 'convergence_q']].mean(axis = 1)
df_strong["avg"] = avg

df_weak = df_weak.sort_values(by='avg')
df_strong = df_strong.sort_values(by='avg')

# take also the max
max_val = df_weak[['convergence_loss', 'convergence_p', 'convergence_q']].max(axis = 1)
df_weak["max"] = max_val
max_val = df_strong[['convergence_loss', 'convergence_p', 'convergence_q']].max(axis = 1)
df_strong["max"] = max_val

C:\Users\feder\AppData\Local\Temp\ipykernel_44752\3162828709.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_weak["avg"] = avg
C:\Users\feder\AppData\Local\Temp\ipykernel_44752\3162828709.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_strong["avg"] = avg


In [26]:
df_weak

,p,q,alpha,convergence_loss,convergence_p,convergence_q,sampler,gamma,avg,max
0,0.17,0.08,0.3,2,5,5,sequential,None,4.0,5
0,0.17,0.08,0.3,2,5,5,neal_seq,10,4.0,5
0,0.17,0.08,0.3,2,5,5,neal_seq,10000,4.0,5
0,0.17,0.08,0.3,2,5,6,neal_seq,0,4.333333,6
0,0.17,0.08,0.3,2,5,6,neal_seq,0.1,4.333333,6
0,0.17,0.08,0.3,2,5,6,neal_seq,1,4.333333,6
0,0.17,0.08,0.3,13,10,9,batched,None,10.666667,13
0,0.17,0.08,0.3,13,10,9,neal_batched,0,10.666667,13
0,0.17,0.08,0.3,13,10,9,neal_batched,0.1,10.666667,13
0,0.17,0.08,0.3,13,10,9,neal_batched,1,10.666667,13


In [27]:
df_strong

,p,q,alpha,convergence_loss,convergence_p,convergence_q,sampler,gamma,avg,max
0,0.4,0.2,0.4,2,3,3,sequential,None,2.666667,3
0,0.4,0.2,0.4,2,3,3,neal_seq,10000,2.666667,3
0,0.4,0.2,0.4,2,4,3,neal_seq,0,3.0,4
0,0.4,0.2,0.4,2,4,3,neal_seq,0.1,3.0,4
0,0.4,0.2,0.4,2,4,3,neal_seq,1,3.0,4
0,0.4,0.2,0.4,2,4,3,neal_seq,10,3.0,4
0,0.4,0.2,0.4,7,6,6,batched,None,6.333333,7
0,0.4,0.2,0.4,7,6,6,neal_batched,0,6.333333,7
0,0.4,0.2,0.4,7,6,6,neal_batched,0.1,6.333333,7
0,0.4,0.2,0.4,7,6,6,neal_batched,1,6.333333,7


In [20]:
extra = False
if extra:
    pq_list = [(0.17, 0.08), (0.4, 0.2)]
    alpha_list = [0.2, 0.3, 0.4]
    n_iter = 150
    burn_in = 0
    np.random.seed(seed)

    convergence_lb = 0.9
    convergence_ub = 1.1

    # convergence iteration (store for each combination the iteration where R_hat is in the interval
    convergence_iter = np.zeros((len(pq_list), len(alpha_list), len(gamma_list)+1))
    # seed
    np.random.seed(0)

    pic = False

    # for over the lists above
    for (p,q) in pq_list:
        for alpha in alpha_list:
            sbm = Homogeneous_SBM(n, k, p, q, Neal=False)
            A = sbm.get_A()
            z = sbm.get_z()
            # run the samplers (n_chains)
            # batched
            batched_p_list = np.zeros((n_iter, n_chains))
            batched_q_list = np.zeros((n_iter, n_chains))
            loss_list = np.zeros((n_iter, n_chains))
            for a in range(n_chains):
                z_init = warm_initializer(z, alpha, n, k)
                sampler = batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, pi_pri = pi)
                sampler.run(n_iter)
                # store the p_lists
                batched_p_list[:, a] = sampler.get_p_list()
                batched_q_list[:, a] = sampler.get_q_list()
                z_list = sampler.get_z_list()
                loss_list[:, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

            # compute R_hat for subsequence (1,r) for all r
            batched_R_hat_p = np.zeros(n_iter//2)
            batched_R_hat_q = np.zeros(n_iter//2)
            idx = np.zeros(n_iter//2)

            for r in range(1+splits, n_iter, 2):
                # take the first r rows of the p_list and q_list
                p_list_split = batched_p_list[:r+1, :]
                q_list_split = batched_q_list[:r+1, :]
                # convert p_list_split from array to list of arrays (n_chains arrays)
                p_list_split = [p_list_split[:, i] for i in range(n_chains)]
                q_list_split = [q_list_split[:, i] for i in range(n_chains)]
                p_list_split = split_chains(p_list_split, splits)
                q_list_split = split_chains(q_list_split, splits)
                batched_R_hat_p[r//2] = R_hat(p_list_split)
                batched_R_hat_q[r//2] = R_hat(q_list_split)
                idx[r//2] = r

            # average the loss over the chains
            loss_avg = np.mean(loss_list, axis=1)

            conv_p = find_convergence(batched_R_hat_p, convergence_lb, convergence_ub)
            conv_q = find_convergence(batched_R_hat_q, convergence_lb, convergence_ub)
            convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), 0] = max(conv_p, conv_q)*2
            print("Convergence iteration: {}".format(convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), 0]))

            print('batched, p: {}, q: {}, alpha: {}'.format(p, q, alpha))
            # plot the R_hat for p and q in the same subplot, and the loss in a separate subplot
            plt.figure(figsize = (15,15))
            plt.subplot(2, 1, 1)
            plt.plot(idx, batched_R_hat_p, label='R_hat_p')
            plt.plot(idx, batched_R_hat_q, label='R_hat_q')
            # add the convergence bounds
            plt.axhline(y=convergence_lb, color='r', linestyle='--')
            plt.axhline(y=convergence_ub, color='r', linestyle='--')
            # vertical line at the convergence iteration
            plt.axvline(x=convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), 0], color='g', linestyle='--')
            plt.title('R_hat for p and q')
            plt.subplot(2, 1, 2)
            plt.plot(loss_avg, label='loss')
            plt.title('loss')
            if pic:
                plt.savefig('speed/p{}_q{}_alpha{}.png'.format(p, q, alpha))
            plt.show()

            for gamma in gamma_list:
                # run the samplers (n_chains)
                # batched
                batched_p_list = np.zeros((n_iter, n_chains))
                batched_q_list = np.zeros((n_iter, n_chains))
                loss_list = np.zeros((n_iter, n_chains))
                for a in range(n_chains):
                    z_init = warm_initializer(z, alpha, n, k)
                    sampler = Neal_batched_Gibbs_sampler(A, z_init, alpha_p, beta_p, alpha_q, beta_q, gamma = gamma)
                    sampler.run(n_iter)
                    # store the p_lists
                    batched_p_list[:, a] = sampler.get_p_list()
                    batched_q_list[:, a] = sampler.get_q_list()
                    z_list = sampler.get_z_list()
                    loss_list[:, a] = [loss(z, z_list[i])[0] for i in range(n_iter)]

                # compute R_hat for subsequence (1,r) for all r
                batched_R_hat_p = np.zeros(n_iter//2)
                batched_R_hat_q = np.zeros(n_iter//2)
                idx = np.zeros(n_iter//2)

                for r in range(1+splits, n_iter, 2):
                    # take the first r rows of the p_list and q_list
                    p_list_split = batched_p_list[:r+1, :]
                    q_list_split = batched_q_list[:r+1, :]
                    # convert p_list_split from array to list of arrays (n_chains arrays)
                    p_list_split = [p_list_split[:, i] for i in range(n_chains)]
                    q_list_split = [q_list_split[:, i] for i in range(n_chains)]
                    p_list_split = split_chains(p_list_split, splits)
                    q_list_split = split_chains(q_list_split, splits)
                    batched_R_hat_p[r//2] = R_hat(p_list_split)
                    batched_R_hat_q[r//2] = R_hat(q_list_split)
                    idx[r//2] = r

                # average the loss over the chains
                loss_avg = np.mean(loss_list, axis=1)
                
                conv_p = find_convergence(batched_R_hat_p, convergence_lb, convergence_ub)
                conv_q = find_convergence(batched_R_hat_q, convergence_lb, convergence_ub)
                convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), gamma_list.index(gamma)+1] = max(conv_p, conv_q)*2
                print("Convergence iteration: {}".format(convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), gamma_list.index(gamma)+1]))

                print("Neal, batched, p: {}, q: {}, alpha: {}, gamma: {}".format(p, q, alpha, gamma))
                # plot the R_hat for p and q in the same subplot, and the loss in a separate subplot
                plt.figure(figsize = (15,15))
                plt.subplot(2, 1, 1)
                plt.plot(idx, batched_R_hat_p, label='R_hat_p')
                plt.plot(idx, batched_R_hat_q, label='R_hat_q')
                # add the convergence bounds
                plt.axhline(y=convergence_lb, color='r', linestyle='--')
                plt.axhline(y=convergence_ub, color='r', linestyle='--')
                # vertical line at the convergence iteration
                plt.axvline(x=convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), gamma_list.index(gamma)+1], color='g', linestyle='--')
                plt.title('R_hat for p and q')
                plt.subplot(2, 1, 2)
                plt.plot(loss_avg, label='loss')
                plt.title('loss')
                if pic:
                    plt.savefig('speed/neal_p{}_q{}_alpha{}_gamma{}.png'.format(p, q, alpha, gamma))
                plt.show()



In [21]:
if extra:
    # for each alpha, plot the convergence iteration as a function of gamma (use vertical lines for each point, not a line)
    pic=False
    for alpha in alpha_list:
        plt.figure(figsize = (15,15))
        for (p,q) in pq_list:
            idx = np.arange(len(gamma_list)+1)
            plt.plot(idx, convergence_iter[pq_list.index((p,q)), alpha_list.index(alpha), :], label='p: {}, q: {}'.format(p,q))
        plt.title('Convergence iteration for alpha = {}'.format(alpha))
        plt.xlabel('gamma')
        plt.ylabel('convergence iteration')
        plt.xticks(idx)
        plt.legend()
        if pic:
            plt.savefig('r_hat/convergence_iteration_alpha{}.png'.format(alpha))
        plt.show()

